# Discovering MNE and MOABB

## MNE
> \[MNE is an\] Open-source Python package for exploring, visualizing, and analyzing human neurophysiological data: MEG, EEG, sEEG, ECoG, NIRS, and more.

More information and documentation can be found at https://mne.tools/stable/index.html


### Downloading a dataset

To discover the features of MNE, we will make use of MOABB to download an SSVEP BCI dataset. MOABB internally uses MNE for data representation, so the downloaded data will be returned in an MNE format.

The dataset used here is the `SSVEPExo` dataset from the University of Versailles [1].

> The datasets contains recording from 12 male and female subjects aged between 20 and 28 years. Informed consent was obtained from all subjects, each one has signed a form attesting her or his consent. The subject sits in an electric wheelchair, his right upper limb is resting on the exoskeleton. The exoskeleton is functional but is not used during the recording of this experiment.
\
A panel of size 20x30 cm is attached on the left side of the chair, with 3 groups of 4 LEDs blinking at different frequencies. Even if the panel is on the left side, the user could see it without moving its head. The subjects were asked to sit comfortably in the wheelchair and to follow the auditory instructions, they could move and blink freely.
\
A sequence of trials is proposed to the user. A trial begin by an audio cue indicating which LED to focus on, or to focus on a fixation point set at an equal distance from all LEDs for the reject class. A trial lasts 5 seconds and there is a 3 second pause between each trial. The evaluation is conducted during a session consisting of 32 trials, with 8 trials for each frequency (13Hz, 17Hz and 21Hz) and 8 trials for the reject class, i.e. when the subject is not focusing on any specific blinking LED.
\
There is between 2 and 5 sessions for each user, recorded on different days, by the same operators, on the same hardware and in the same conditions.

https://neurotechx.github.io/moabb/generated/moabb.datasets.SSVEPExo.html#moabb.datasets.SSVEPExo

In [ ]:
from moabb.datasets import SSVEPExo

dataset = SSVEPExo()
dataset.download()
dataset.get_data()

Let's create a variable `raw` containing the raw, unprocessed data from the first subject in this experiment. MNE gives a nice overview of the metadata of this EEG recording.

In [ ]:
subj,session,run = 1, 'session_0', 'run_0'
raw = dataset.get_data(subjects=[subj])[subj][session][run]
raw

### Inspect raw data

MNE offers a lot of useful tools for data manipulation and visualization. You can plot the locations of the elctrodes used to record the data:

In [ ]:
sphere=(0,-25,0,100)
_ = raw.plot_sensors(show_names=True, kind='topomap', sphere=sphere)

Plot the raw EEG data:

In [ ]:
_ = raw.plot(duration=4, color={'eeg':'darkblue'}, scalings=dict(eeg=1e-2))

Or plot the power spectral density (psd). The psd applies a Fourier transform and visualizes the frequency content of the EEG signal. Notice the large peak at 50Hz (and its harmonic at 100=50\*2Hz) caused by noise from the power grid alternating at 50Hz, and the increase in activity from 8-12Hz indicating alpha activity in the EEG. Generally, the psd has a *1/f* characteristic, meaning that lower frequencies will have relatively higher power regardless of specific relevant brain activity occurring at these frequencies. Because no preprocessing was applied here, the SSVEP stimulation frequencies are not yet clearly visible from the psd.

In [ ]:
_ = raw.plot_psd(sphere=sphere)

Finally, to make preprocessing and classification easier, MNE can extract events that are recorded in a stimulation channel in the EEG. For SSVEP, these events signify the onsets of the stimulations of different targets. Events can also be used as classes for classifying the three SSVEP different frequencies.

In [ ]:
import numpy as np
from mne import pick_types

stim_idc = pick_types(raw.info, eeg=False, stim=True)
[raw.ch_names[i] for i in stim_idc]

In [ ]:
from mne import find_events
from mne.viz import plot_events

events = find_events(raw)
events

In [ ]:
_= plot_events(events)

## MOABB

The Mother Of All BCI Benchmarks is a package for interfacing with various EEG-BCI datasets from multiple paradigms and defines a framework to benchmark algorithms.

More information and documentation can be found at https://neurotechx.github.io/moabb/

### Paradigm

MOABB can implement standardized preprocessing pipelines that can be applied to multiple datasets to fairly compare these in a benchmark. MOABB calls a preprocessing pipeline a paradigm.

In [ ]:
from moabb.paradigms import SSVEP
dataset.event_id= {
    '13': 2,
    '17': 4,
    '21': 3,    
    'rest': 1,
}
paradigm = SSVEP(fmin=10, fmax=25, n_classes=3)

### Pipelines

Classification algorithms can equally be defined independent of the dataset and are implemented using [Scikit-Learn pipelines](https://scikit-learn.org/stable/modules/compose.html#pipeline) to allow for multiple feature extraction, transformation and classification steps.

In [ ]:
from sklearn.pipeline import make_pipeline
from moabb.pipelines import SSVEP_CCA


interval = dataset.interval
freqs = paradigm.used_events(dataset)

pipelines = dict()
pipelines["CCA"] = make_pipeline(
    SSVEP_CCA(interval=interval, freqs=freqs, n_harmonics=3)
)

### Evaluation

Finally, when we have defined a paradigm and some pipelines, we can run the benchmark and evaluate BCI algorithm performance.

In [ ]:
from moabb.evaluations import WithinSessionEvaluation


evaluation = WithinSessionEvaluation(
    paradigm=paradigm,
    datasets=dataset,
    suffix='ssvep_workshop_moabb_intro',
    overwrite=True
)
results = evaluation.process(pipelines)
results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure()
sns.catplot(kind='bar', x="score", y="subject", hue=None, data=results)

## References

[1] Emmanuel K. Kalunga, Sylvain Chevallier, Quentin Barthelemy. “Online SSVEP-based BCI using Riemannian Geometry”. Neurocomputing, 2016. arXiv report: https://arxiv.org/abs/1501.03227